# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [3]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [4]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../weather_database/WeatherPy_database.csv")

# Display sample data
vacation_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,60.66,55,75,18.41,broken clouds
1,1,Watsa,CD,3.0372,29.5355,66.88,94,99,2.04,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,73.65,52,0,14.72,clear sky
3,3,Kushiro,JP,42.9750,144.3747,40.64,70,97,6.29,overcast clouds
4,4,Vaini,TO,-21.2000,-175.2000,78.96,89,75,6.91,broken clouds
5,5,Togul,RU,53.4650,85.9128,-5.75,79,13,0.58,few clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,68.56,90,89,4.97,overcast clouds
7,7,Saldanha,ZA,-33.0117,17.9442,74.75,45,13,0.78,few clouds
8,8,Micheweni,TZ,-4.9667,39.8333,82.13,71,13,0.69,light rain
9,9,Mataura,NZ,-46.1927,168.8643,59.49,79,100,3.36,overcast clouds


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [6]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [16]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Hofn"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Hofn"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Husavik"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Grindavik"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Vestmannaeyjar"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [17]:
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index = True)

# Display sample data
itinerary_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,502,Hofn,IS,64.2539,-15.2082,35.87,89,100,15.59,overcast clouds
1,391,Husavik,IS,66.0449,-17.3389,32.36,84,71,9.98,broken clouds
2,512,Grindavik,IS,63.8424,-22.4338,36.07,69,20,8.05,few clouds
3,222,Vestmannaeyjar,IS,63.4427,-20.2734,37.33,69,3,22.21,clear sky
4,502,Hofn,IS,64.2539,-15.2082,35.87,89,100,15.59,overcast clouds


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [20]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]]

# Display sample data
waypoints_df

,Lat,Lng
0,64.2539,-15.2082
1,66.0449,-17.3389
2,63.8424,-22.4338
3,63.4427,-20.2734
4,64.2539,-15.2082


## Use GeoViews to create map that shows the four cities in the itinerary

In [27]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Current Description"]
)

In [28]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [29]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [30]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '03a88608310946b7bcea3779e742bee0',
 'waypoints': '64.2539,-15.2082|66.0449,-17.3389|63.8424,-22.4338|63.4427,-20.2734|64.2539,-15.2082'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [31]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [32]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [34]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [35]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df["longitude"] = longitude
route_df["latitude"] = latitude

# Display sample data
route_df.head()

,longitude,latitude
0,-15.207847,64.254248
1,-15.207630,64.254347
2,-15.208216,64.254590
3,-15.207991,64.254692
4,-15.207894,64.254736


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [36]:
# Configure the route path by using the GeoViews' Path function
route_path = route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "red"
)

In [49]:
# Display a composed plot by using the route_map and route_path object
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Description)
   .Path.I   :Path   [longitude,latitude]